### Configurações Pyspark

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

### Configurações Pandas

In [7]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [8]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched')
           )

### Procedure Occurrence

In [9]:
window_procedure_occurrence = Window.partitionBy().orderBy('person_id', 'dtnasc_sinasc')

In [10]:
df_procedure_occurrence = (df_input
                         .withColumn('person_id', F.col('person_id_infant'))
                         .withColumn('procedure_concept_id', F.when(F.col('parto_sinasc').isin(0, 88, 99), F.lit(0))
                                                              .when(F.col('parto_sinasc')==1, F.lit(44784097))
                                                              .when(F.col('parto_sinasc')==2, F.lit(4015701))
                                                              .otherwise(0))
                           .withColumn('procedure_date', F.col('dtnasc_sinasc'))
                        .withColumn('procedure_datetime', F.lit(None))
                        .withColumn('procedure_end_date', F.lit(None))
                        .withColumn('procedure_end_datetime', F.lit(None))
                        .withColumn('procedure_type_concept_id', F.lit(32879))
                        .withColumn('modifier_concept_id', F.lit(None))
                        .withColumn('quantity', F.lit(None))
                        .withColumn('provider_id', F.lit(None))
                        .withColumn('visit_occurrence_id', F.col('visit_occurrence_id'))
                        .withColumn('visit_detail_id', F.lit(None))
                        .withColumn('procedure_source_value', F.col('parto_sinasc'))
                        .withColumn('procedure_source_concept_id', F.lit(None))
                        .withColumn('modifier_source_value', F.lit(None))
                           .withColumn('procedure_occurrence_id', F.row_number().over(window_procedure_occurrence)+F.lit(1000000000))
                          ).select(F.col('procedure_occurrence_id').cast('integer'),
                                    F.col('person_id').cast('integer'),
                                    F.col('procedure_concept_id').cast('integer'),
                                    F.col('procedure_date').cast('date'),
                                    F.col('procedure_datetime').cast('timestamp'),
                                    F.col('procedure_end_date').cast('date'),
                                    F.col('procedure_end_datetime').cast('timestamp'),
                                    F.col('procedure_type_concept_id').cast('integer'),
                                    F.col('modifier_concept_id').cast('integer'),
                                    F.col('quantity').cast('float'),
                                    F.col('provider_id').cast('integer'),
                                    F.col('visit_occurrence_id').cast('integer'),
                                    F.col('visit_detail_id').cast('integer'),
                                    F.col('procedure_source_value').cast('string'),
                                    F.col('procedure_source_concept_id').cast('integer'),
                                    F.col('modifier_source_value').cast('string')
                                )

df_procedure_occurrence.count()

16590554

In [11]:
write_to_postgresl(df_procedure_occurrence, tb_name='tuberculose.procedure_occurrence', write_mode='append')

25/07/17 16:37:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/17 16:37:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/17 16:37:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/17 16:37:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/17 16:37:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


'Total time: 3:10:07.534437 and - Total rows: 16590554 - Total columns: 16'